# Chapter 1: Getting Started

**Data-Juicer User Guide**

- Git Commit: `v1.0.5`
- Commit Date: 2026-01-16
- Repository: https://github.com/datajuicer/data-juicer

---

# Table of Contents

1. [Install Data-Juicer](#install-data-juicer)
2. [Create Sample JSONL Data](#create-sample-jsonl-data)
3. [Write Basic YAML Config](#write-basic-yaml-config)
4. [Execute Pipeline](#execute-pipeline)
5. [Check Output](#check-output)
6. [Learning Path](#learning-path)
   - [Core Concepts (Recommended Order)](#core-concepts-recommended-order)
   - [Advanced Topics (Appendices)](#advanced-topics-appendices)

## Install Data-Juicer

Data-Juicer can be easily installed via pip. We recommend using `uv` for faster installation, but standard `pip` works too.

Detailed installation tutorial [here](https://datajuicer.github.io/data-juicer/en/main/docs/tutorial/Installation.html)

In [ ]:
!uv pip install py-data-juicer

## Create Sample JSONL Data

Data-Juicer works with JSONL (JSON Lines) format, where each line is a valid JSON object. This format is efficient for streaming large datasets and is widely used in the ML community.

In [ ]:
import json
import os

# Create data directory
os.makedirs('./data', exist_ok=True)

# Sample data
samples = [
    {"text": "Today is Sunday and it's a happy day!", "meta": {"src": "web", "date": "2024-01-01"}},
    {"text": "Do you need a cup of coffee?", "meta": {"src": "social", "author": "user123"}},
    {"text": "Machine learning is transforming the world.", "meta": {"src": "article"}},
    {"text": "Short.", "meta": {"src": "web"}},
    {"text": "This is a longer text with more content to demonstrate filtering capabilities.", "meta": {"src": "blog"}}
]

# Write JSONL file
with open('./data/sample.jsonl', 'w') as f:
    for sample in samples:
        f.write(json.dumps(sample) + '\n')

print(f"Created sample dataset with {len(samples)} samples")

## Write Basic YAML Config

Data-Juicer uses YAML configuration files ("recipes") to define processing pipelines. A recipe specifies:
- **Input/Output paths**: Where to read and write data
- **Processing operators**: What transformations to apply
- **Execution settings**: Parallelism, caching, etc.

Let's create a simple recipe that filters text by length and language, then removes duplicates.

In [ ]:
config = """# Basic Data-Juicer Configuration
project_name: 'getting_started'

# Input/Output paths
dataset_path: './data/sample.jsonl'
export_path: './outputs/processed.jsonl'

# Number of parallel processes
np: 1

# Processing pipeline
process:
  # 1. Filter by text length
  - text_length_filter:
      min_len: 10
      max_len: 200
  
  # 2. Filter by language (English)
  - language_id_score_filter:
      lang: 'en'
      min_score: 0.8
  
  # 3. Remove duplicates
  - document_deduplicator:
      lowercase: true
"""

# Save config
os.makedirs('./configs', exist_ok=True)
with open('./configs/basic.yaml', 'w') as f:
    f.write(config)

print("Config saved to ./configs/basic.yaml")

## Execute Pipeline

Data-Juicer provides two ways to run pipelines:
1. **Command-line**: Using the `dj-process` command
2. **Programmatic**: Using Python API for more control

Both methods produce identical results. Choose based on your workflow preference.

In [ ]:
!dj-process --config ./configs/basic.yaml

or running programmatically

In [ ]:
from data_juicer.config import init_configs
cfg = init_configs(['--config', './configs/basic.yaml'])

In [ ]:
from data_juicer.core import DefaultExecutor

# Create and run executor
executor = DefaultExecutor(cfg)
dataset = executor.run()

print("\nProcessed samples:")
for i, sample in enumerate(dataset):
    print(f"{i+1}. {sample['text']}")

## Check Output

Let's verify the results. Notice how the pipeline filtered out:
- Short texts (< 10 characters)
- Non-English texts (language score < 0.8)
- Duplicate entries

In [ ]:
import json

# Read processed data
with open('./outputs/processed.jsonl', 'r') as f:
    processed = [json.loads(line) for line in f]

print(f"Original samples: 5")
print(f"Processed samples: {len(processed)}")
print("\nProcessed data:")
for i, sample in enumerate(processed, 1):
    print(f"\n{i}. {sample['text'][:60]}...")

## Learning Path

Here's a recommended learning path through the remaining chapters:

### Core Concepts (Recommended Order)

1. **[Chapter 2: Building Recipes](./02_Building_Recipes.ipynb)**
   - Understand recipe structure (global parameters, process pipeline, operator parameters)
   - Create basic and custom recipes
   - Override parameters via CLI
   - Explore pre-defined recipes from the Recipe Gallery (data-juicer-hub)

2. **[Chapter 3: Data Formats](./03_Data_Formats.ipynb)**
   - Learn Data-Juicer's unified format (DJ Format)
   - Convert between dialog formats (Messages, ShareGPT, Alpaca, Query-Response)
   - Handle multimodal format conversion (LLaVA, MMC4, InternVid, etc.)

3. **[Chapter 4: Operators Usage](./04_Operators_Usage.ipynb)**
   - Use operators programmatically via Python API
   - Chain operators sequentially or batch process
   - Inspect operator statistics

4. **[Chapter 5: Analysis & Visualization](./05_Analysis_and_Visualization.ipynb)**
   - Run data analysis with `dj-analyze`
   - Interpret statistics and visualizations
   - Compare datasets before and after processing

5. **[Chapter 6: Distributed Processing with Ray](./06_Distributed_Processing_with_Ray.ipynb)**
   - Set up Ray clusters (local and multi-node)
   - Use demo configs from `demos/process_on_ray/`
   - Monitor resources via Ray Dashboard
   - Run distributed deduplication

### Advanced Topics

- **[Chapter 7: Pre-processing](./07_Preprocessing.ipynb)**
  - Split datasets by language
  - Convert raw formats (arXiv, Stack Exchange) to JSONL
  - Serialize complex metadata fields

- **[Chapter 8: Multimodal Data Processing](./08_Multimodal_Data_Processing.ipynb)**
  - Understand multimodal format with special tokens
  - Process image-text, video-text, audio-text data
  - Convert between multimodal formats (LLaVA, Video-ChatGPT, WavCaps, etc.)
  - Apply multimodal operators (image/video/audio filters)

- **[Chapter 9: Advanced Dataset Configuration](./09_Advanced_Dataset_Configuration.ipynb)**
  - Mix multiple datasets with custom weights
  - Sample subsets from large datasets
  - Map custom field names to standard fields
  - Load remote datasets from HuggingFace

## Additional Resources

- **Documentation**: https://datajuicer.github.io/data-juicer
- **GitHub**: https://github.com/datajuicer/data-juicer
- **Recipe Gallery**: https://datajuicer.github.io/data-juicer-hub
- **Operator Reference**: https://datajuicer.github.io/data-juicer/en/main/docs/Operators.html